In [17]:
import os
import matplotlib.pyplot as plt 
# Data handling
import pandas as pd
import numpy as np
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import seaborn as sns
from matplotlib import animation
from IPython.display import HTML
from matplotlib import rc
import pylab
import matplotlib.patches as mpatches
from bokeh.palettes import Spectral6
from bokeh.models import CustomJS, Slider

In [18]:
# Bokeh Libraries
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Div
from bokeh.layouts import gridplot, column

# Output to file
output_file('phi-gm-linked-stats.html',
                title='76ers Game Log')

wd = os.getcwd()
og = pd.read_csv("CaseStudy.csv", encoding='latin-1')

df_fin = og.drop(columns = ['race_ethnicity:confidence','_unit_state','_golden','_last_judgment_at', 'birthplace:confidence', 'date_of_birth:confidence','religion',
'religion:confidence','sexual_orientation','sexual_orientation:confidence','race_ethnicity_gold','religion_gold','sexual_orientation_gold','year_of_award_gold',
'biourl', 'birthplace_gold','date_of_birth_gold', 'birthplace_gold','date_of_birth_gold', 'race_ethnicity_gold','religion_gold','sexual_orientation_gold', 'birthplace_gold','date_of_birth_gold','_trusted_judgments', 'year_of_award:confidence'])

In [19]:
df_fin['_unit_id'] = 0

df_fin = df_fin.drop_duplicates()

In [20]:
df_white = df_fin[df_fin['race_ethnicity'] == 'White']
df_not_white = df_fin[df_fin['race_ethnicity'] != 'White']

In [21]:
df_bokeh = df_fin.groupby(['race_ethnicity','award']).count()

In [22]:
df_bokeh = df_bokeh.reset_index()

In [23]:
df_bokeh = df_bokeh.drop(columns = ['birthplace','date_of_birth','movie','person','year_of_award'])

In [24]:
df_bokeh_cds =  ColumnDataSource(data = df_bokeh)

In [25]:
group = df_fin.groupby(['race_ethnicity', 'award'])
source = ColumnDataSource(group)

In [26]:
df_bokeh

,race_ethnicity,award,_unit_id
0,Asian,Best Director,2
1,Asian,Best Supporting Actor,1
2,Asian,Best Supporting Actress,1
3,Black,Best Actor,4
4,Black,Best Supporting Actor,4
5,Black,Best Supporting Actress,6
6,Hispanic,Best Actor,1
7,Hispanic,Best Supporting Actor,4
8,Hispanic,Best Supporting Actress,3
9,Middle Eastern,Best Actor,1


In [27]:
df_bokeh.loc[df_bokeh['award'] == 'Best Actor', 'Num'] = 1
df_bokeh.loc[df_bokeh['award'] == 'Best Actress', 'Num'] = 2
df_bokeh.loc[df_bokeh['award'] == 'Best Director', 'Num'] = 3
df_bokeh.loc[df_bokeh['award'] == 'Best Supporting Actor', 'Num'] = 4
df_bokeh.loc[df_bokeh['award'] == 'Best Supporting Actress', 'Num'] = 5

In [28]:
df_bokeh

,race_ethnicity,award,_unit_id,Num
0,Asian,Best Director,2,3.0
1,Asian,Best Supporting Actor,1,4.0
2,Asian,Best Supporting Actress,1,5.0
3,Black,Best Actor,4,1.0
4,Black,Best Supporting Actor,4,4.0
5,Black,Best Supporting Actress,6,5.0
6,Hispanic,Best Actor,1,1.0
7,Hispanic,Best Supporting Actor,4,4.0
8,Hispanic,Best Supporting Actress,3,5.0
9,Middle Eastern,Best Actor,1,1.0


In [29]:
from bokeh.models import HoverTool
from bokeh.io import curdoc
from bokeh.layouts import widgetbox
from bokeh.models import Slider

In [30]:
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, GroupFilter
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
import bokeh 

rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)

source = ColumnDataSource(df_bokeh)
view1 = CDSView(source=source, filters=[GroupFilter(column_name='race_ethnicity', group='Black')])
view11 = CDSView(source=source, filters=[GroupFilter(column_name='race_ethnicity', group='White')])
view2 = CDSView(source=source, filters=[GroupFilter(column_name='race_ethnicity', group='Asian')])
view3 = CDSView(source=source, filters=[GroupFilter(column_name='race_ethnicity', group='Hispanic')])
view4 = CDSView(source=source, filters=[GroupFilter(column_name='race_ethnicity', group='Middle Eastern')])
view5 = CDSView(source=source, filters=[GroupFilter(column_name='race_ethnicity', group='Multiracial')])

plot_size_and_tools = {'plot_height': 400, 'plot_width': 800,
                        'tools':['box_select', 'reset', 'help']}
y = [1,2,3,4,5,7,8,9,10]
mapper = linear_cmap(field_name='_unit_id', palette=Spectral6 ,low=min(y) ,high=max(y))

p1 = figure(title="All Ethnicities", **plot_size_and_tools)
p1.circle(x='Num', y = '_unit_id', source=source, color=mapper, alpha = 0.7, size = 10)
tab1 = Panel(child=p1, title="Full")
p1.xaxis.major_label_overrides = {1: 'Best Actor', 2: 'Best Actress', 3: 'Best Director', 4: 'Best Supporting Actor', 5: 'Best Supporting Actress'}
p1.xgrid.grid_line_color = None
p1.ygrid.band_fill_alpha = 0.1
p1.ygrid.band_fill_color = "lightgrey"
hover = HoverTool(tooltips=[('Number of awards','@_unit_id')])
p1.add_tools(hover)


p11 = figure(title="Ethnicity: White", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools)
p11.circle(x='Num', y='_unit_id', source=source, view=view11, color=mapper, size = 10, alpha = 0.9)
tab11 = Panel(child=p11, title="White")
p11.xaxis.major_label_overrides = {1: 'Best Actor', 2: 'Best Actress', 3: 'Best Director', 4: 'Best Supporting Actor', 5: 'Best Supporting Actress'}
p11.xgrid.grid_line_color = None
p11.ygrid.band_fill_color = "lightgrey"
hover = HoverTool(tooltips=[('Number of awards','@_unit_id')])
p11.add_tools(hover)

p2 = figure(title="Ethnicity: Black", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools)
p2.circle(x='Num', y='_unit_id', source=source, view=view1, color=mapper, size = 10, alpha = 0.9)
tab2 = Panel(child=p2, title="Black")
p2.xaxis.major_label_overrides = {1: 'Best Actor', 2: 'Best Actress', 3: 'Best Director', 4: 'Best Supporting Actor', 5: 'Best Supporting Actress'}
p2.xgrid.grid_line_color = None
p2.ygrid.band_fill_color = "lightgrey"
hover = HoverTool(tooltips=[('Number of awards','@_unit_id')])
p2.add_tools(hover)

p3 = figure(title="Ethnicity: Asian", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools)
p3.circle(x='Num', y='_unit_id', source=source, view=view2, color=mapper, size = 10, alpha = 0.9)
tab3 = Panel(child=p3, title="Asian")
p3.xaxis.major_label_overrides = {1: 'Best Actor', 2: 'Best Actress', 3: 'Best Director', 4: 'Best Supporting Actor', 5: 'Best Supporting Actress'}
p3.xgrid.grid_line_color = None
p3.ygrid.band_fill_color = "lightgrey"
hover = HoverTool(tooltips=[('Number of awards','@_unit_id')])
p3.add_tools(hover)

p4 = figure(title="Ethnicity: Hispanic", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools)
p4.circle(x='Num', y='_unit_id', source=source, view=view3, color=mapper, size = 10, alpha = 0.9)
tab4 = Panel(child=p4, title="Hispanic")
p4.xaxis.major_label_overrides = {1: 'Best Actor', 2: 'Best Actress', 3: 'Best Director', 4: 'Best Supporting Actor', 5: 'Best Supporting Actress'}
p4.xgrid.grid_line_color = None
p4.ygrid.band_fill_color = "lightgrey"
hover = HoverTool(tooltips=[('Number of awards','@_unit_id')])
p4.add_tools(hover)

p5 = figure(title="Ethnicity: Middle Eastern", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools)
p5.circle(x='Num', y='_unit_id', source=source, view=view4, color=mapper, size = 10, alpha = 0.9)
tab5 = Panel(child=p5, title="Middle Eastern")
p5.xaxis.major_label_overrides = {1: 'Best Actor', 2: 'Best Actress', 3: 'Best Director', 4: 'Best Supporting Actor', 5: 'Best Supporting Actress'}
p5.xgrid.grid_line_color = None
p5.ygrid.band_fill_color = "lightgrey"
hover = HoverTool(tooltips=[('Number of awards','@_unit_id')])
p5.add_tools(hover)

p6 = figure(title="Ethnicity: Multiracial", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools)
p6.circle(x='Num', y='_unit_id', source=source, view=view5, color=mapper, size = 10, alpha = 0.9)
tab6 = Panel(child=p6, title="Multiracial")
p6.xaxis.major_label_overrides = {1: 'Best Actor', 2: 'Best Actress', 3: 'Best Director', 4: 'Best Supporting Actor', 5: 'Best Supporting Actress'}
p6.xgrid.grid_line_color = None
p6.ygrid.band_fill_color = "lightgrey"
hover = HoverTool(tooltips=[('Number of awards','@_unit_id')])
p6.add_tools(hover)


show(Tabs(tabs=[tab1, tab11, tab2, tab3, tab4, tab5, tab6]))

In [16]:
df_bokeh

,race_ethnicity,award,_unit_id,Num
0,Asian,Best Director,2,3.0
1,Asian,Best Supporting Actor,1,4.0
2,Asian,Best Supporting Actress,1,5.0
3,Black,Best Actor,4,1.0
4,Black,Best Supporting Actor,4,4.0
5,Black,Best Supporting Actress,6,5.0
6,Hispanic,Best Actor,1,1.0
7,Hispanic,Best Supporting Actor,4,4.0
8,Hispanic,Best Supporting Actress,3,5.0
9,Middle Eastern,Best Actor,1,1.0
